## Hyperparameter tuning
Use cases: check if windows are closed if raining or when leaving house

Still following up with the model tuning of the binary classification problem of open/closed windows

I will apply hyperparameter tuning and compare the results on tensor board

In [173]:
!pip install keras-tuner
!pip install tensorboard

In [174]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
# libraries for displaying images
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import shutil
import math
from skimage import exposure
from tensorflow.keras.callbacks import CSVLogger

import keras_tuner as kt
from keras_tuner import HyperParameters, BayesianOptimization

# Start TensorBoard
%tensorboard --logdir ./log

In [175]:
WIDTH = 224
HEIGHT = 224
NUM_CLASSES = 2
TRAIN_DATA_DIR = r"../contents/windows_imgs/train"
VAL_DATA_DIR = r"../contents/windows_imgs/val"
DATA_DIR = r"../contents/windows_imgs/data"
TEST_DATA_DIR = r"../contents/windows_imgs/test"
LOG_HPARM = './log/hparm'
LOG_DIR = './log'
TRAIN_SAMPLES = 9
VAL_SAMPLES = 4
BATCH_SIZE = 32
EPOCH = 100
CLASS_THRESHOLD = 0.5
LEARNING_RATE = 0.001

from datetime import datetime
ts = datetime.now()
ts = f"{ts:%Y%m%d-%H%M%S}"


shutil.rmtree(LOG_DIR, ignore_errors=True)

### Prepare the data
load image data from directory and preprocess it

In [176]:

from numpy import left_shift, right_shift


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    # width_shift_range = 0.1,
    # height_shift_range = 0.1,
    zoom_range=0.1,
    fill_mode = 'nearest',
    brightness_range=(0.8,1.2),
    # shear_range=0.2,
    # featurewise_center =True,
    # horizontal_flip=True,
    # validation_split=0.1
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [177]:

train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=(WIDTH, HEIGHT),
    batch_size = BATCH_SIZE,
    shuffle = True,
    seed = 12345,
    class_mode = 'binary',
    # subset='training',
    save_to_dir=r"../contents/windows_imgs/output/train/",
    save_prefix="",
    save_format='jpg')
TRAIN_SAMPLES = train_generator.samples

val_generator = val_datagen.flow_from_directory(
    VAL_DATA_DIR,
    target_size=(WIDTH, HEIGHT),
    batch_size = BATCH_SIZE,
    shuffle = False,
    class_mode = 'binary',
    # subset='validation',
    save_to_dir=r"../contents/windows_imgs/output/val/",
    save_prefix="",
    save_format='jpg')
VAL_SAMPLES = val_generator.samples


Found 34 images belonging to 2 classes.
Found 32 images belonging to 2 classes.


Code to show the augmented images

In [178]:
# # define the figure plot area
# # fig, ax = plt.subplots(1,NUM_IMG, figsize=(12,4))
# import matplotlib.pyplot as plt

# for i in range(BATCH_SIZE):
#     print(f'Run #{i}')
#     img, label = train_generator.next()
#     # classval = np.argmax(label[0], axis = 0)    # convert one-hot encoding to index
#     # classkey = find_key(classval, iter_img.class_indices) # get class key from index value
#     plt.imshow(img[0])
#     plt.show()
#     # print(f'img shape {img.shape}')
#     # print(f'img label {classkey}\n')

# plt.show()

In [179]:
# import cv2

# # cv2.waitKey(0)
# cam = cv2.VideoCapture(0)

# result, cam_img = cam.read()
# if result:
#     # cv2.imshow("Captured image", cam_img)
#     cv2.imwrite(TEST_DATA_DIR + r"\test_window.jpeg", cam_img)
#     # cv2.waitKey(0)
#     # cv2.destroyAllWindows()
# else:
#     print("No image detected. Please try again.") 


# from sympy import plot


# img_path = r'../contents/windows_imgs/test/test_window.jpeg'
# img = image.load_img(img_path, target_size=(224, 224))
# img_array = image.img_to_array(img)
# expanded_img_array = np.expand_dims(img_array, axis=0)
# preprocessed_img = expanded_img_array / 255.  # Preprocess the image
# prediction = model.predict(preprocessed_img)
# print(prediction)
# print(val_generator.class_indices)

# # plt.figure(1,1)
# plt.imshow(img)
# plt.show()

#### Load the pre-trained model and add in fine tuning layers
Now we define the model layers to freeze, discard to classification layers
then finally add in the fine tuning layers


In [180]:

def model_maker(hp):
    # num_layers = hp.Int('num_layers', min_value=1, max_value=2)
    lr = hp.Choice('learning_rate', values=[1e-3, 1e-4])
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    # act = hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'])

    custom_model = Sequential()
    # load model
    base_model = MobileNet(
        include_top=False,
        input_shape=(WIDTH, HEIGHT,3))
    
    for layer in base_model.layers[:]:
        layer.trainable = False

    custom_model.add(base_model)
    custom_model.add(GlobalAveragePooling2D())
    # custom_model.add(Flatten())

    # for num in range(num_layers):
    custom_model.add(Dense(hp.Int('units', min_value=128, max_value=1024, step=256), activation = 'relu'))
    custom_model.add(Dropout(hp.Float('dropout', min_value=0.3, max_value=0.6, step=0.1)))

    custom_model.add(Dense(1, activation='sigmoid'))

    custom_model.compile(
        loss =  'binary_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])

    return custom_model


Just trying out with another pre-trained model

In [181]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2


def model_maker_inceptionResNetV2():
    # load model
    base_model = InceptionResNetV2(
        include_top=False,
        input_shape=(WIDTH, HEIGHT,3))
    for layer in base_model.layers[:]:
        layer.trainable = False
    input = Input(shape=(WIDTH, HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Flatten()(custom_model)
    custom_model = Dense(256, activation = 'relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(1, activation='sigmoid')(custom_model)
    return Model(inputs=input, outputs=predictions)


Testing with another model from scratch, however it perform poorly. I am not sure why

In [182]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
# from tensorflow.keras.optimizers import SGD,RMSprop,adam

def model_maker_scratch():
    from_scratch = Sequential()
    from_scratch.add(Convolution2D(224, 3,3,input_shape=(WIDTH, HEIGHT, 3)))
    from_scratch.add(Activation('relu'))
    from_scratch.add(Convolution2D(1000, 3, 3))
    from_scratch.add(Activation('relu'))
    from_scratch.add(MaxPooling2D(pool_size=(2, 2)))
    from_scratch.add(Dropout(0.5))
    # from_scratch.add(Convolution2D(64, 3, 3))
    from_scratch.add(Convolution2D(1000, 3, 3))
    from_scratch.add(Activation('relu'))
    from_scratch.add(MaxPooling2D(pool_size=(2, 2)))
    from_scratch.add(Dropout(0.5))
    # 

    from_scratch.add(Dense(256, activation='relu'))
    from_scratch.add(Dropout(0.5))
    from_scratch.add(Dense(1, activation='sigmoid'))
    # from_scratch.add(Dense(1))
    # from_scratch.add(Activation('sigmoid'))
    return(from_scratch)


In [183]:
#Allow TensorBoard callbacks
tensorboard_callback = tf.keras.callbacks.TensorBoard(f"{LOG_DIR}\{ts}",
                                                      histogram_freq=1,
                                                      write_images=True)

csv_logger = CSVLogger('windows-training-' + 'log.csv',
                        append=True,
                        separator=';')



Hyperparameters configuration

In [184]:
tuner = kt.BayesianOptimization(
	model_maker,
	objective="val_accuracy",
	max_trials=20,
	seed=42,
	directory=LOG_HPARM)

tuner.search(
	train_generator,
	validation_data=val_generator,
	batch_size=BATCH_SIZE,
	epochs=10
)

Trial 16 Complete [00h 00m 35s]
val_accuracy: 0.65625

Best val_accuracy So Far: 0.6875
Total elapsed time: 00h 08m 51s

Search: Running Trial #17

Value             |Best Value So Far |Hyperparameter
0.001             |0.001             |learning_rate
128               |640               |units
0.3               |0.3               |dropout

Epoch 1/10


KeyboardInterrupt: 

In [ ]:

# tuner.search(train_data=train_generator, epochs=10, validation_data=val_generator)
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best learning rate: {best_hps.get('learning_rate')}")
print(f"Best number of units: {best_hps.get('units')}")
best_model = tuner.hypermodel.build(best_hps)

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor = 'val_accuracy', min_delta=0.001, patience=10)

best_model.fit(
    train_generator,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES)/BATCH_SIZE),
    epochs=EPOCH,
    callbacks=[tensorboard_callback,csv_logger,early_stopping],
    validation_data=val_generator,
    validation_steps=math.ceil(float(VAL_SAMPLES)/BATCH_SIZE)
    )


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a 'history' object with training and validation loss
# Example: history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

# Retrieve training and validation loss
train_loss = best_model.history.history['loss'] 
val_loss = best_model.history.history['val_loss']

# Create a plot
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(train_loss) + 1), train_loss, label='Training Loss', marker='o')
plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss', marker='x')

# Customize the plot
plt.title('Training vs. Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

# Show the plot
plt.show()

In [ ]:
h5_path = f'..\contents\models\winclass_{ts}.h5'
best_model.save(h5_path)
tf.saved_model.save(best_model, "tmp/model/1/")

import shutil

# Specify the file you want to copy
source_file = h5_path
destination_file = 'model.h5'

# Copy the file
shutil.copyfile(source_file, destination_file)

print(f"File copied successfully from {source_file} to {destination_file}")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

model = load_model('model.h5')

In [ ]:
import cv2

# cv2.waitKey(0)
cam = cv2.VideoCapture(0)

result, cam_img = cam.read()
if result:
    
    # cv2.imshow("Captured image", cam_img)
    # equ_img = cv2.equalizeHist(cam_img)
    cv2.imwrite(TEST_DATA_DIR + r"\test_window.jpeg", cam_img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
else:
    print("No image detected. Please try again.") 

from skimage import exposure, transform
from skimage.io import imread

img_path = r'../contents/windows_imgs/test/test_window.jpeg'
img = image.load_img(img_path, target_size=(224, 224))

# img = imread(img_path)
# img = exposure.equalize_hist(img)
# new_shape = (224, 224)
# img = transform.resize(img, new_shape, anti_aliasing=True)

img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.  # Preprocess the image
prediction = model.predict(preprocessed_img)
print(prediction)
print(val_generator.class_indices)

# plt.figure(1,1)
plt.title(f"File Name: {img_path}, \n Prediction: Window is  {'Close' if prediction < CLASS_THRESHOLD else 'Open'}")
plt.imshow(img)
plt.show()    


In [ ]:
from skimage import exposure, transform
from skimage.io import imread

img_path = r'../contents/windows_imgs/test/test_window.jpeg'
img = image.load_img(img_path, target_size=(224, 224))

# img = imread(img_path)
# img = exposure.equalize_hist(img)
# new_shape = (224, 224)
# img = transform.resize(img, new_shape, anti_aliasing=True)

img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.  # Preprocess the image
prediction = model.predict(preprocessed_img)
print(prediction)
print(val_generator.class_indices)

# plt.figure(1,1)
plt.title(f"File Name: {img_path}, \n Prediction: Window is  {'Close' if prediction < CLASS_THRESHOLD else 'Open'}")
plt.imshow(img)
plt.show()    


## List all images in test folder and make predictions

In [ ]:
import os
filelist = os.listdir(VAL_DATA_DIR + r"/close/")
# filelist = os.listdir(TEST_DATA_DIR)
jpegfiles = [file for file in filelist if file.endswith('.jpeg')]


Wanted to shrink the images down and display it inside a grid here.. work in progress

In [ ]:
# ## loop through file list and make a grid of images
# cols = 3
# rows = -(-jpegfiles.count // cols)
# fig, axes = plt.subplot(3, rows, figsize(18,6))
# axes = axes.flatten()

# # Plot each image in the corresponding subplot
# for i, image in enumerate(images):
#     img = image.load_img(os.path.join(VAL_DATA_DIR  + r"/close/",file), target_size=(WIDTH,HEIGHT))
#     # img = image.load_img(os.path.join(TEST_DATA_DIR,file), target_size=(WIDTH,HEIGHT))
#     img_array = image.img_to_array(img)
#     expanded_img_array = np.expand_dims(img_array, axis=0)
#     preprocessed_img = expanded_img_array / 255.  # Preprocess the image
#     prediction = model.predict(preprocessed_img)
#     print(prediction)
#     print(val_generator.class_indices)

#     ax = axes[i]
#     ax.imshow(img)
#     ax.title(f"File Name: {file}, \n Prediction: Window is  {'Close' if prediction < CLASS_THRESHOLD else 'Open'}")

#     ax.axis('off')  # Turn off axis labels

# plt.figure()

In [ ]:
for file in jpegfiles:
    img = image.load_img(os.path.join(VAL_DATA_DIR  + r"/close/",file), target_size=(WIDTH,HEIGHT))
    # img = image.load_img(os.path.join(TEST_DATA_DIR,file), target_size=(WIDTH,HEIGHT))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = expanded_img_array / 255.  # Preprocess the image
    prediction = model.predict(preprocessed_img)
    print(prediction)
    print(val_generator.class_indices)

    plt.title(f"File Name: {file}, \n Prediction: Window is  {'Close' if prediction < CLASS_THRESHOLD else 'Open'}")
    plt.imshow(img)
    plt.show()    


    